In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 64kB/s 
     |████████████████████████████████| 204kB 43.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=4ba8933e491f05ed5cd720e3e21361f11e09ac0306a0f4cf16a25135b4395da3
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 43 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Primer Cuatrimestre de 2018. Examen parcial, primera oportunidad
## 1)  Nintendo of America (EEUU) tiene información de ventas de videojuegos físicas mensuales totalizadas en EEUU las cuales se realizan en cadenas de tiendas de videojuegos en el siguiente RDD: (id_videojuego, id_tienda, mes, anio, total_ventas_mensuales).  
## Por otro lado tenemos un RDD con información de las tiendas y de su ubicación (id_tienda, direccion, latitud, longitud, codigo_postal, estado). 
 
 
## Con esta información escribir un programa en pySpark para obtener la tienda que realizó menor cantidad de ventas en el estado de “Georgia” en todo el año 2019.   (***) (15 pts) 

In [21]:
tiendas = [(100,'7085 Prairie Drive 231',491,371,'georgia'),
(103,'Dawsonville GA 30534 179',471,345,'arkansas'),
(103,'8433 Pheasant Street 260',216,386,'florida'),
(103,'Mason OH 45040 332',238,789,'georgia'),
(104,'9 Ridgewood Rd 179',303,483,'nevada'),
(105,'Saratoga Springs NY 12866 160',134,377,'new york'),
(106,'490 Canal St 451',443,283,'massachussets'),
(107,'Stone Mountain GA 30083 406',299,779,'arkansas'),
(108, '913 Elizabeth St 254',434,797,'georgia'),
(111,'Manassas VA 20109 372',338,698,'georgia'),
(112,'24 Logan Dr 390',160,519,'arkansas'),
(113,'Pawtucket RI 02860 222',329,478,'georgia'),
(114,'593 Cypress R 165',170,417,'florida'),
(114,'Fairborn OH 45324 182',351,703,'massachussets'),
(115,'2 Spring Street 112',325,790,'georgia'),
(116,'Portland ME 04103 105',305,321,'georgia'),
(117,'307 Trenton Road 257',138,635,'georgia'),
(119,'Shelton CT 06484 337',306,338,'georgia'),
(120,'9670 South Oakwood Drive 198',343,465,'massachussets')]

ventas = [(32690,115,8,2019,967),
(31638,106,3,2018,485),
(10280,104,2,2018,780),
(35355,108,10,2018,939),
(22623,112,3,2019,696),
(12383,103,3,2019,912),
(7922,105,8,2019,248),
(35819,117,4,2018,502),
(36189,116,8,2018,915),
(35859,119,4,2018,524),
(26952,116,2,2019,161),
(29231,120,7,2019,703),
(46451,107,8,2019,926),
(17093,111,12,2018,992),
(21571,104,8,2018,485),
(12514,103,3,2019,347),
(22284,113,7,2019,801),
(23759,100,10,2019,415),
(8553,106,4,2019,611),
(2603,100,7,2018,213),
(11037,103,10,2019,103),
(14086,114,5,2019,253),
(27595,108,12,2019,747),
(46181,113,9,2018,223),
(4312,103,6,2018,805),
(29959,114,7,2018,101),
(7648,112,1,2019,160),
(2714,119,3,2019,301),
(44643,120,2,2019,871),
(30926,114,1,2019,488)]

In [22]:
ventasRDD = sc.parallelize(ventas)
tiendasRDD = sc.parallelize(tiendas)

In [23]:
ventasRDD.collect()

[(32690, 115, 8, 2019, 967),
 (31638, 106, 3, 2018, 485),
 (10280, 104, 2, 2018, 780),
 (35355, 108, 10, 2018, 939),
 (22623, 112, 3, 2019, 696),
 (12383, 103, 3, 2019, 912),
 (7922, 105, 8, 2019, 248),
 (35819, 117, 4, 2018, 502),
 (36189, 116, 8, 2018, 915),
 (35859, 119, 4, 2018, 524),
 (26952, 116, 2, 2019, 161),
 (29231, 120, 7, 2019, 703),
 (46451, 107, 8, 2019, 926),
 (17093, 111, 12, 2018, 992),
 (21571, 104, 8, 2018, 485),
 (12514, 103, 3, 2019, 347),
 (22284, 113, 7, 2019, 801),
 (23759, 100, 10, 2019, 415),
 (8553, 106, 4, 2019, 611),
 (2603, 100, 7, 2018, 213),
 (11037, 103, 10, 2019, 103),
 (14086, 114, 5, 2019, 253),
 (27595, 108, 12, 2019, 747),
 (46181, 113, 9, 2018, 223),
 (4312, 103, 6, 2018, 805),
 (29959, 114, 7, 2018, 101),
 (7648, 112, 1, 2019, 160),
 (2714, 119, 3, 2019, 301),
 (44643, 120, 2, 2019, 871),
 (30926, 114, 1, 2019, 488)]

In [24]:
tiendasRDD.collect()

[(100, '7085 Prairie Drive 231', 491, 371, 'georgia'),
 (103, 'Dawsonville GA 30534 179', 471, 345, 'arkansas'),
 (103, '8433 Pheasant Street 260', 216, 386, 'florida'),
 (103, 'Mason OH 45040 332', 238, 789, 'georgia'),
 (104, '9 Ridgewood Rd 179', 303, 483, 'nevada'),
 (105, 'Saratoga Springs NY 12866 160', 134, 377, 'new york'),
 (106, '490 Canal St 451', 443, 283, 'massachussets'),
 (107, 'Stone Mountain GA 30083 406', 299, 779, 'arkansas'),
 (108, '913 Elizabeth St 254', 434, 797, 'georgia'),
 (111, 'Manassas VA 20109 372', 338, 698, 'georgia'),
 (112, '24 Logan Dr 390', 160, 519, 'arkansas'),
 (113, 'Pawtucket RI 02860 222', 329, 478, 'georgia'),
 (114, '593 Cypress R 165', 170, 417, 'florida'),
 (114, 'Fairborn OH 45324 182', 351, 703, 'massachussets'),
 (115, '2 Spring Street 112', 325, 790, 'georgia'),
 (116, 'Portland ME 04103 105', 305, 321, 'georgia'),
 (117, '307 Trenton Road 257', 138, 635, 'georgia'),
 (119, 'Shelton CT 06484 337', 306, 338, 'georgia'),
 (120, '9670 Sout

In [28]:
# Nos quedamos unicamente con las tiendas de georgia
tiendas_georgia = (tiendasRDD.map(lambda x: (x[0],x[4]))).filter(lambda x: x[1]=='georgia')
tiendas_georgia.collect()

[(100, 'georgia'),
 (103, 'georgia'),
 (108, 'georgia'),
 (111, 'georgia'),
 (113, 'georgia'),
 (115, 'georgia'),
 (116, 'georgia'),
 (117, 'georgia'),
 (119, 'georgia')]

In [40]:
# Nos quedamos unicamente con las ventas del anio 2019
ventasRDD_short = (ventasRDD.filter(lambda x: x[3]==2019 )).map(lambda x: (x[1],x[4])).reduceByKey(lambda x,y: x+y)
ventasRDD_short.collect()

[(112, 856),
 (116, 161),
 (120, 1574),
 (100, 415),
 (106, 611),
 (114, 741),
 (108, 747),
 (115, 967),
 (103, 1362),
 (105, 248),
 (107, 926),
 (113, 801),
 (119, 301)]

In [41]:
# unifamos los RDDs y ordenamos de menor a mayor por cantidad de ventas
ventas_georgia_por_tienda = (tiendas_georgia.join(ventasRDD_short)).map(lambda x: (x[0],x[1][1])).sortBy(lambda x: x[1])
ventas_georgia_por_tienda.collect()

[(116, 161),
 (119, 301),
 (100, 415),
 (108, 747),
 (113, 801),
 (115, 967),
 (103, 1362)]

In [42]:
print('La tienda en Georgia que vendio menos videojuegos fisicos es la tienda', ventas_georgia_por_tienda.take(1))

La tienda en Georgia que vendio menos videojuegos fisicos es la tienda [(116, 161)]


# FIN DEL EJERCICIO